<a href="https://colab.research.google.com/github/RohanVipin/GENAI-/blob/main/NeuralinkLectures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch datasets accelerate streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

os.environ['HF_DATASETS_CACHE']= '/content/drive/MyDrive/hf_cache'


ValueError: Mountpoint must not already contain files

In [ ]:
from datasets import load_dataset

dataset = load_dataset("UKPLab/liar")
print(dataset)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Repo card metadata block was not found. Setting CardData to empty.


DatasetDict({
    train: Dataset({
        features: ['text', 'label_text', 'labels', 'context'],
        num_rows: 10269
    })
    validation: Dataset({
        features: ['text', 'label_text', 'labels', 'context'],
        num_rows: 1284
    })
    test: Dataset({
        features: ['text', 'label_text', 'labels', 'context'],
        num_rows: 1283
    })
})


In [ ]:
#preparing training data and test data
import pandas as pd
fake_train = dataset['train'].filter(lambda ex:ex['labels']==1)
fake_val = dataset['validation'].filter(lambda ex:ex['labels']==1)
df = pd.DataFrame(fake_train)
df.head()

Filter:   0%|          | 0/10269 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1284 [00:00<?, ? examples/s]

,text,label_text,labels,context
0,Dwayne Bohac says the Annies List political gr...,false statement,1,a mailer
1,Blog Posting says Health care reform legislati...,false statement,1,a news release
2,Mitt Romney says When Mitt Romney was governor...,false statement,1,an interview with CBN News
3,Dennis Richardson says Women and men both are ...,false statement,1,a campaign debate
4,Nancy Pelosi says I dont know who (Jonathan Gr...,false statement,1,a news conference


In [ ]:
#tokenize for gpt2
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(examples):
    encodings= tokenizer(examples['text'],truncation=True,padding='max_length',max_length=64)
    encodings['labels']=encodings['input_ids']
    return encodings

train_dataset = fake_train.map(tokenize_fn,batched=True)
val_dataset = fake_val.map(tokenize_fn,batched=True)

train_dataset.set_format(type='torch',columns=['input_ids','attention_mask','labels'])
val_dataset.set_format(type='torch',columns=['input_ids','attention_mask','labels'])


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/2840 [00:00<?, ? examples/s]

Map:   0%|          | 0/379 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel

model=GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [ ]:
#setting training arguments

from transformers import Trainer,TrainingArguments
training_args = TrainingArguments(
    output_dir = './gpt2-fake-news',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    report_to='none'
)

trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)
trainer.train()

# save the fine tuned model
trainer.save_model('./gpt2-fake-news')
tokenizer.save_pretrained('./gpt2-fake-news')

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,1.586000
200,1.378900
300,1.357400
400,1.404200
500,1.355100
600,1.322900
700,1.382500
800,1.410400
900,1.313200
1000,1.390800


('./gpt2-fake-news/tokenizer_config.json',
 './gpt2-fake-news/special_tokens_map.json',
 './gpt2-fake-news/vocab.json',
 './gpt2-fake-news/merges.txt',
 './gpt2-fake-news/added_tokens.json')

In [ ]:
model.save_pretrained('/content/drive/MyDrive/gpt2-fake-news')
tokenizer.save_pretrained('/content/drive/MyDrive/gpt2-fake-news')

('/content/drive/MyDrive/gpt2-fake-news/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2-fake-news/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2-fake-news/vocab.txt',
 '/content/drive/MyDrive/gpt2-fake-news/added_tokens.json',
 '/content/drive/MyDrive/gpt2-fake-news/tokenizer.json')

In [4]:
#this is for reloading the model for later (no retraining required)
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/gpt2-fake-news')
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/gpt2-fake-news')
model.to("cuda" if torch.cuda.is_available() else "cpu")


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [5]:
#testiing the model

def genearte_fake_news(prompt):
  inputs = tokenizer(prompt,return_tensors='pt')
  inputs = {k:v.to(model.device) for k,v in inputs.items()}

  outputs = model.generate(
      **inputs,
      max_length=200,
      do_sample=True,
      top_k=50,
      top_p=0.95,
      temperature=0.8,
      pad_token_id=tokenizer.eos_token_id,
      early_stopping = False
  )
  return tokenizer.decode(outputs[0],skip_special_tokens=True)


In [6]:
import gradio as gr

iface = gr.Interface(
    fn=genearte_fake_news,
    inputs=gr.Textbox(lines=2, placeholder="Enter a prompt, e.g., 'The government announced...'"),
    outputs="text",
    title="Fake News Generator",
    description="Enter a prompt to generate fake news using GPT-2"
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://58357b883b85c78ed7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:

# -------------------------------
# STEP 2: Mount Google Drive
# -------------------------------
from google.colab import drive
drive.mount('/content/drive')

# Set working directory inside Drive (Change path if you like)
import os
project_path = "/content/drive/MyDrive/fake_news_project"
os.makedirs(project_path, exist_ok=True)
os.chdir(project_path)

# -------------------------------
# STEP 3: Load dataset
# -------------------------------
import pandas as pd

# Make sure Fake.csv and True.csv are in the project_path folder in Drive
fake_df = pd.read_csv("Fake.csv")
true_df = pd.read_csv("True.csv")

fake_df['label'] = 1  # Fake
true_df['label'] = 0  # Real

df = pd.concat([fake_df, true_df], ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)  # Shuffle

# Check column names
print(df.columns)

# Handle cases where 'text' column might be missing
if 'text' in df.columns:
    df['final_text'] = df['text']
elif 'content' in df.columns:
    df['final_text'] = df['content']
else:
    # Combine title + text if both exist
    if 'title' in df.columns and 'text' in df.columns:
        df['final_text'] = df['title'] + " " + df['text']
    elif 'title' in df.columns:
        df['final_text'] = df['title']
    else:
        raise ValueError("No valid text column found in dataset!")

# -------------------------------
# STEP 4: Train-Test Split
# -------------------------------
from sklearn.model_selection import train_test_split

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['final_text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42
)

# -------------------------------
# STEP 5: Tokenization
# -------------------------------
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

import torch
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} | {'labels': torch.tensor(self.labels[idx])}

train_dataset = NewsDataset(train_encodings, train_labels)
test_dataset = NewsDataset(test_encodings, test_labels)

# -------------------------------
# STEP 6: Load Model
# -------------------------------
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)











Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index(['title', 'text', 'subject', 'date', 'label'], dtype='object')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [1]:
# -------------------------------
# STEP 7: Training Setup
# -------------------------------
from transformers import Trainer, TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="steps",
    save_steps=500,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,  # Start small to save GPU time
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True
)

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {'accuracy': acc, 'f1': f1, 'precision': precision, 'recall': recall}

from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# -------------------------------
# STEP 8: Train Model
# -------------------------------
trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


NameError: name 'model' is not defined

In [ ]:
# -------------------------------
# STEP 9: Save Model to Drive
# -------------------------------
save_path = project_path + "/fake_news_model"
trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
# -------------------------------
# STEP 10: Gradio UI
# -------------------------------
import gradio as gr
from transformers import pipeline

classifier = pipeline("text-classification", model=save_path, tokenizer=tokenizer, device=0)  # GPU

def predict_news(text):
    result = classifier(text)[0]
    label = "Fake News" if result['label'] == 'LABEL_1' else "Real News"
    confidence = round(result['score'] * 100, 2)
    return f"{label} (Confidence: {confidence}%)"

iface = gr.Interface(
    fn=predict_news,
    inputs=gr.Textbox(lines=10, placeholder="Paste news article here..."),
    outputs="text",
    title="Fake News Detector (DistilBERT)",
    description="Enter a news article to check if it's Fake or Real using DistilBERT."
)

iface.launch(share=True)